In [11]:
# Import pandas
import pandas as pd
import numpy as np

# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from pathlib import Path

import tensorflow as tf

# Import Gradio
import gradio as gr

In [12]:
# Set the model's file path
model_path = Path("milking_the_data.keras")

# Load the model to a new object
model = tf.keras.models.load_model(model_path)
model

<Sequential name=sequential_1, built=True>

In [13]:
import pickle

# Load the scaler from the file
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [14]:
def predict_milk_production(lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month):
    # Prepare the input data for the model
    input_data = np.array([[lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month]])
    #scale the data
    input_data_scaled = scaler.transform(input_data)
    # Make prediction
    prediction = model.predict(input_data_scaled)
    
    return prediction[0][0]  # Return the predicted value


In [15]:
# Define the Gradio interface
# Define the Gradio interface
inputs = [
    gr.Number(label='Lactation Number (lact_num)'),
    gr.Number(label='Average THI 3 days (avg_thi3)'),
    gr.Number(label='Average CSI 3 days (avg_csi3)'),
    gr.Number(label='Average CSI 30 days (avg_csi30)'),
    gr.Number(label='Average Precipitation Spring (avg_precspring)'),
    gr.Number(label='Date Difference (date_diff)'),
    gr.Number(label='Calving Month (calv_month)'),
    gr.Number(label='Calving Day of Month (calv_day_of_month)'),
    gr.Number(label='Alping Month (alp_month)'),
    gr.Number(label='Alping Day of Month (alp_day_of_month)')
]

outputs = gr.Textbox(label='Predicted Milk Production')

interface = gr.Interface(fn=predict_milk_production, inputs=inputs, outputs=outputs)

# Launch the interface
# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


C:\Users\jclib\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
